In [1]:
import pandas as pd
import numpy as np

In [62]:
import time

In [4]:
df_trans = pd.read_csv('df_trans_mccfin.csv')
df_trans

,customer_id,transaction_month,transaction_day,transaction_amt,merchant_mcc
0,855115,7,3,1500,Супермаркеты
1,997036,6,6,0,Супермаркеты
2,398237,5,24,2500,Рестораны
3,997036,6,2,0,Супермаркеты
4,291636,7,25,0,Супермаркеты
...,...,...,...,...,...
3951094,153986,6,30,29500,Дом и ремонт
3951095,346407,6,18,500,Дом и ремонт
3951096,358612,6,6,2500,Одежда и обувь
3951097,346407,6,7,500,Супермаркеты


In [8]:
data = df_trans.groupby(['customer_id'], as_index = False).aggregate({'transaction_amt': 'sum','transaction_month':'min'})
data.transaction_month = data.transaction_month - 4
data.transaction_amt = data.transaction_amt / data.transaction_month
data2 = df_trans.groupby(['customer_id','merchant_mcc'], as_index = False).aggregate({'transaction_amt': 'sum','transaction_day':'count'}).rename(columns = {'transaction_amt':'trat','transaction_day':'coun'})
id = list(data.customer_id)
data = data.drop ('transaction_month', 1)

In [9]:
data

,customer_id,transaction_amt
0,15,2221000.0
1,73,1149500.0
2,91,423500.0
3,144,5000.0
4,150,0.0
...,...,...
46943,1098683,30000.0
46944,1099266,17000.0
46945,1099662,9500.0
46946,1099955,70000.0


In [10]:
data2

,customer_id,merchant_mcc,trat,coun
0,15,Авиабилеты,1210500,2
1,15,Аптеки,8000,4
2,15,Дом и ремонт,7500,5
3,15,Дьюти-фри,2000,1
4,15,Книги,11500,1
...,...,...,...,...
474095,1103495,Развлечения,2000,1
474096,1103495,Страховые услуги,31500,1
474097,1103495,Супермаркеты,12500,8
474098,1103495,Топливо,9500,4


In [20]:
i=id[0]
trat=data2.loc[(data2.customer_id == i)].sort_values(by='trat', ascending=False)
coun=data2.loc[(data2.customer_id == i)].sort_values(by='coun', ascending=False)
trat

,customer_id,merchant_mcc,trat,coun
0,15,Авиабилеты,1210500,2
8,15,Отели,344500,2
7,15,Одежда и обувь,247500,11
15,15,Супермаркеты,198500,38
17,15,Фастфуд,50500,2
11,15,Развлечения,39000,11
13,15,Спорттовары,33000,2
12,15,Рестораны,27500,8
16,15,Транспорт,15000,1
10,15,Прочие магазины,14000,2


In [23]:
trat.iloc[2]['merchant_mcc']

'Одежда и обувь'

In [28]:
for i in id:
    trat=data2.loc[(data2.customer_id == i)].sort_values(by='trat', ascending=False)
    coun=data2.loc[(data2.customer_id == i)].sort_values(by='coun', ascending=False)
    data.loc[data.customer_id==i, 'BZ1'] = trat.iloc[0]['merchant_mcc']
    data.loc[data.customer_id==i, 'BC1'] = coun.iloc[0]['merchant_mcc']
    if len(trat) > 1:
        data.loc[data.customer_id==i, 'BZ2'] = trat.iloc[1]['merchant_mcc']
        data.loc[data.customer_id==i, 'BC2'] = coun.iloc[1]['merchant_mcc']
    else:
        data.loc[data.customer_id==i, 'BZ2'] = 'NONE'
        data.loc[data.customer_id==i, 'BC2'] = 'NONE'
    if len(trat) > 2:
        data.loc[data.customer_id==i, 'BZ3'] = trat.iloc[2]['merchant_mcc']
        data.loc[data.customer_id==i, 'BC3'] = coun.iloc[2]['merchant_mcc']
    else:
        data.loc[data.customer_id==i, 'BZ3'] = 'NONE'
        data.loc[data.customer_id==i, 'BC3'] = 'NONE'

In [36]:
data.head(30)

,customer_id,transaction_amt,BZ1,BZ2,BZ3,BC1,BC2,BC3
0,15,2.221000e+06,Авиабилеты,Отели,Одежда и обувь,Супермаркеты,Развлечения,Одежда и обувь
1,73,1.149500e+06,Авиабилеты,Рестораны,Отели,Каршеринг,Супермаркеты,Дом и ремонт
2,91,4.235000e+05,Дом и ремонт,Прочие магазины,Отели,Дом и ремонт,Супермаркеты,Топливо
3,144,5.000000e+03,Аптеки,Топливо,Супермаркеты,Супермаркеты,Топливо,Аптеки
4,150,0.000000e+00,Супермаркеты,NONE,NONE,Супермаркеты,NONE,NONE
5,164,1.000000e+03,Фастфуд,NONE,NONE,Фастфуд,NONE,NONE
6,178,1.870000e+05,Консультационные услуги,Красота,Супермаркеты,Фастфуд,Супермаркеты,Транспорт
7,183,1.500000e+04,Супермаркеты,Фастфуд,Одежда и обувь,Дом и ремонт,Одежда и обувь,Прочие магазины
8,192,7.500000e+02,Одежда и обувь,Супермаркеты,NONE,Одежда и обувь,Супермаркеты,NONE
9,362,1.475000e+05,Супермаркеты,Дом и ремонт,Одежда и обувь,Супермаркеты,Фастфуд,Аптеки


In [26]:
trat=data2.loc[(data2.customer_id == i)].sort_values(by='trat', ascending=False)
coun=data2.loc[(data2.customer_id == i)].sort_values(by='coun', ascending=False)
trat

,customer_id,merchant_mcc,trat,coun
66,150,Супермаркеты,0,4


In [30]:
data.to_csv('123.csv', index=False)

In [32]:
data = pd.read_csv('123.csv')

In [34]:
data3 = df_trans.groupby(['customer_id'], as_index = False).aggregate({'transaction_amt': 'sum'})

In [37]:
data3.head(30)

,customer_id,transaction_amt
0,15,2221000
1,73,1149500
2,91,423500
3,144,5000
4,150,0
5,164,1000
6,178,187000
7,183,15000
8,192,1500
9,362,147500


In [39]:
data['sum'] = data3['transaction_amt']
data.head(50)

,customer_id,transaction_amt,BZ1,BZ2,BZ3,BC1,BC2,BC3,sum
0,15,2.221000e+06,Авиабилеты,Отели,Одежда и обувь,Супермаркеты,Развлечения,Одежда и обувь,2221000
1,73,1.149500e+06,Авиабилеты,Рестораны,Отели,Каршеринг,Супермаркеты,Дом и ремонт,1149500
2,91,4.235000e+05,Дом и ремонт,Прочие магазины,Отели,Дом и ремонт,Супермаркеты,Топливо,423500
3,144,5.000000e+03,Аптеки,Топливо,Супермаркеты,Супермаркеты,Топливо,Аптеки,5000
4,150,0.000000e+00,Супермаркеты,NONE,NONE,Супермаркеты,NONE,NONE,0
5,164,1.000000e+03,Фастфуд,NONE,NONE,Фастфуд,NONE,NONE,1000
6,178,1.870000e+05,Консультационные услуги,Красота,Супермаркеты,Фастфуд,Супермаркеты,Транспорт,187000
7,183,1.500000e+04,Супермаркеты,Фастфуд,Одежда и обувь,Дом и ремонт,Одежда и обувь,Прочие магазины,15000
8,192,7.500000e+02,Одежда и обувь,Супермаркеты,NONE,Одежда и обувь,Супермаркеты,NONE,1500
9,362,1.475000e+05,Супермаркеты,Дом и ремонт,Одежда и обувь,Супермаркеты,Фастфуд,Аптеки,147500


In [40]:
data2

,customer_id,merchant_mcc,trat,coun
0,15,Авиабилеты,1210500,2
1,15,Аптеки,8000,4
2,15,Дом и ремонт,7500,5
3,15,Дьюти-фри,2000,1
4,15,Книги,11500,1
...,...,...,...,...
474095,1103495,Развлечения,2000,1
474096,1103495,Страховые услуги,31500,1
474097,1103495,Супермаркеты,12500,8
474098,1103495,Топливо,9500,4


In [41]:
a=data2.loc[(data2.customer_id == i)]
a

,customer_id,merchant_mcc,trat,coun
474090,1103495,Автоуслуги,12500,2
474091,1103495,Дом и ремонт,10500,2
474092,1103495,Красота,6500,2
474093,1103495,Одежда и обувь,33000,9
474094,1103495,Прочие магазины,1500,2
474095,1103495,Развлечения,2000,1
474096,1103495,Страховые услуги,31500,1
474097,1103495,Супермаркеты,12500,8
474098,1103495,Топливо,9500,4
474099,1103495,Фастфуд,1000,3


In [43]:
mcc=list(df_trans.groupby(['merchant_mcc'], as_index = False).aggregate({'transaction_amt': 'sum'}).merchant_mcc)

In [82]:
t1 = time.perf_counter()
for i in id:
    d=data2.loc[(data2.customer_id == i)]
    s = data.loc[(data.customer_id == i)].iloc[0]['sum']
    for m in mcc:
        if len(d.loc[(d.merchant_mcc == m)]) > 0:
            data.loc[data.customer_id==i, m] = d.loc[(d.merchant_mcc == m)].iloc[0]['trat']/s
        else:
            data.loc[data.customer_id==i, m] = 0
t2 = time.perf_counter()

c:\users\екатерина\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in longlong_scalars
  import sys


In [56]:
i=id[0]
m=mcc[0]
d=data2.loc[(data2.customer_id == i)]
d.loc[(data2.merchant_mcc == m)]

,customer_id,merchant_mcc,trat,coun
0,15,Авиабилеты,1210500,2


In [57]:
d.loc[(data2.merchant_mcc == m)].iloc[0]['trat']

1210500

In [59]:
s = data.loc[(data.customer_id == i)].iloc[0]['sum']
s

2221000

In [60]:
d.loc[(data2.merchant_mcc == m)].iloc[0]['trat']/s

0.545024763619991

In [63]:
time.perf_counter()

6802.1988669

In [64]:
time.perf_counter()

6806.7481168

['Авиабилеты',
 'Автоуслуги',
 'Агенства по перевозкам',
 'Агенства по трудоустройству',
 'Агенты недвижимости',
 'Азартные игры',
 'Аптеки',
 'Аренда авто',
 'Ассоциации',
 'Бизнес услуги',
 'Благотворительность',
 'Бухгалтерский учет',
 'Внутригосударственные закупки',
 'Государственные услуги',
 'Детективные и охранные агенства',
 'Дом и ремонт',
 'Дьюти-фри',
 'Железнодорожные билеты',
 'Животные',
 'Здоровье',
 'Знакомства',
 'Информационна-поисковые услуги',
 'Искусство',
 'Канцтовары',
 'Каршеринг',
 'Квази-Кэш',
 'Кино',
 'Книги',
 'Консультационные услуги',
 'Красота',
 'Кредитные бюро',
 'Лизинговые услуги',
 'Ломбарды',
 'Магазины кожаных изделий',
 'Магазины печатающих устройств',
 'Музыка',
 'Налоговые платежи',
 'Образование',
 'Одежда и обувь',
 'От двери до двери',
 'Отели',
 'ПМ Входящий телемаркетинг',
 'ПМ Другое',
 'ПМ Комбинированный каталог',
 'ПМ Продажа по подписке',
 'ПМ Торговля по каталогам',
 'Пополнение небанковских карт',
 'Почтовые услуги',
 'Программиров

In [81]:
t1 = time.perf_counter()
d=data2.loc[(data2.customer_id == i)]
s = data.loc[(data.customer_id == i)].iloc[0]['sum']
for m in mcc:
    if len(d.loc[(d.merchant_mcc == m)]) > 0:
        data.loc[data.customer_id==i, m] = d.loc[(d.merchant_mcc == m)].iloc[0]['trat']/s
    else:
        data.loc[data.customer_id==i, m] = 0
t2 = time.perf_counter()

,customer_id,transaction_amt,BZ1,BZ2,BZ3,BC1,BC2,BC3,sum,Авиабилеты,...,Услуги покупок,Услуги ухода за детьми,Фастфуд,Фото и видео,Фотостудии,Цветы,Цифровые товары,Членские организации,Штрафы,Юридические услуги
0,15,2221000.0,Авиабилеты,Отели,Одежда и обувь,Супермаркеты,Развлечения,Одежда и обувь,2221000,0.545025,...,0.0,0.000000,0.022738,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
1,73,1149500.0,Авиабилеты,Рестораны,Отели,Каршеринг,Супермаркеты,Дом и ремонт,1149500,0.212701,...,0.0,0.001305,0.005220,0.0,0.0,0.0,0.0,0.0,0.011309,0.0
2,91,423500.0,Дом и ремонт,Прочие магазины,Отели,Дом и ремонт,Супермаркеты,Топливо,423500,0.060213,...,0.0,0.000000,0.015348,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
3,144,5000.0,Аптеки,Топливо,Супермаркеты,Супермаркеты,Топливо,Аптеки,5000,0.000000,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
4,150,0.0,Супермаркеты,NONE,NONE,Супермаркеты,NONE,NONE,0,0.000000,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46943,1098683,30000.0,Супермаркеты,Топливо,Дом и ремонт,Супермаркеты,Топливо,Рестораны,30000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46944,1099266,17000.0,Фастфуд,Супермаркеты,Аптеки,Супермаркеты,Цифровые товары,Аптеки,17000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46945,1099662,9500.0,Супермаркеты,Одежда и обувь,Дом и ремонт,Супермаркеты,Прочие магазины,Транспорт,9500,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46946,1099955,70000.0,Спорттовары,Аптеки,Дом и ремонт,Аптеки,Супермаркеты,Здоровье,70000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [70]:
t2-t1

0.17634110000017245

In [71]:
writer = pd.ExcelWriter('sho.xlsx') 
data.to_excel(writer) 
writer.save()

In [73]:
data.loc[(data.customer_id == 15)].iloc[0][9]

0.545024763619991

In [79]:

per = list(range(9, 90))
sss = 0
for ss in per:
    sss+=data.loc[(data.customer_id == 73)].iloc[0][ss]

In [80]:
sss

1.0

In [83]:
data = pd.read_csv('12345.csv')

FileNotFoundError: [Errno 2] File b'12345.csv' does not exist: b'12345.csv'

In [84]:
data.to_csv('12345.csv', index=False)

In [85]:
data

,customer_id,transaction_amt,BZ1,BZ2,BZ3,BC1,BC2,BC3,sum,Авиабилеты,...,Услуги покупок,Услуги ухода за детьми,Фастфуд,Фото и видео,Фотостудии,Цветы,Цифровые товары,Членские организации,Штрафы,Юридические услуги
0,15,2221000.0,Авиабилеты,Отели,Одежда и обувь,Супермаркеты,Развлечения,Одежда и обувь,2221000,0.545025,...,0.0,0.000000,0.022738,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0
1,73,1149500.0,Авиабилеты,Рестораны,Отели,Каршеринг,Супермаркеты,Дом и ремонт,1149500,0.212701,...,0.0,0.001305,0.005220,0.0,0.0,0.0,0.000000,0.0,0.011309,0.0
2,91,423500.0,Дом и ремонт,Прочие магазины,Отели,Дом и ремонт,Супермаркеты,Топливо,423500,0.060213,...,0.0,0.000000,0.015348,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0
3,144,5000.0,Аптеки,Топливо,Супермаркеты,Супермаркеты,Топливо,Аптеки,5000,0.000000,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0
4,150,0.0,Супермаркеты,NONE,NONE,Супермаркеты,NONE,NONE,0,0.000000,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46943,1098683,30000.0,Супермаркеты,Топливо,Дом и ремонт,Супермаркеты,Топливо,Рестораны,30000,0.000000,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0
46944,1099266,17000.0,Фастфуд,Супермаркеты,Аптеки,Супермаркеты,Цифровые товары,Аптеки,17000,0.000000,...,0.0,0.000000,0.382353,0.0,0.0,0.0,0.058824,0.0,0.000000,0.0
46945,1099662,9500.0,Супермаркеты,Одежда и обувь,Дом и ремонт,Супермаркеты,Прочие магазины,Транспорт,9500,0.000000,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0
46946,1099955,70000.0,Спорттовары,Аптеки,Дом и ремонт,Аптеки,Супермаркеты,Здоровье,70000,0.000000,...,0.0,0.000000,0.007143,0.0,0.0,0.0,0.078571,0.0,0.000000,0.0


In [86]:
(t2-t1)/60

134.01278465666667